# 本单元用于记录LeetCode每日一题

## 问题1：[HTML 实体解析器](https://leetcode.cn/problems/html-entity-parser/description/)
### 题目
字符串替换, 将字符串中的特殊符号, 根据规则转化为指定符号。

### 分析
- 使用正则表达式进行划分, 由于我们发现待替换的字符串中都是“&xxx;”的形式，因此我们使用正则表达式进行分组
- re.split(r"(&[^&;]+;)", text):
    - [^&;]: 表示匹配不包括&和;符号
    - +: 表示前面的匹配至少出现一次
    - [^&;]+: 表示必须匹配一个非&和非;的字符
    - (): 表示将内部匹配到的东西作为一个分组整体返回(保证了分割字符中包含有&;) 

### 优化
- 直接使用replace(目标值, 更换值)进行替换即可


In [ ]:
import re
def process(text):
    word_dict = {"&quot;":"\"","&apos;":"\'","&amp;":"&", "&gt;":">","&lt;":"<","&frasl;":"/"}
    word_list = re.split(r"(&[^&;]+;)", text)
    print(word_list)
    for index, word in enumerate(word_list):
        if word in word_dict:
            word_list[index] = word_dict[word]
    return "".join(word_list)

text = "and I quote: &quot;...&quot;"
process(text)


## 问题1：[]()
### 题目


### 分析


### 注意


### 优化


In [1]:
import bisect
a = [1,7,7,8,10]
bisect.bisect_left(a,7)
a

[1, 7, 7, 8, 10]

In [ ]:
import bisect
def process(arr):
    stack = []
    for i in arr:
        

## 问题1：[]()
### 题目


### 分析


### 注意


### 优化
